In [1]:
import layoutparser as lp
import pymupdf
from PIL import Image
import cv2
import numpy as np

import faiss
import torch
from transformers import CLIPProcessor, CLIPModel

device = 'gpu' if torch.cuda.is_available() else 'cpu'

c:\Users\lewis\OneDrive\Desktop\Schoolwork\FYP\FYProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Unified class
class DocumentAnalysis():
    def __init__(self):
        self.model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_R_50_FPN_3x/config', 
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
        self.ocr_agent = lp.TesseractAgent(languages='eng') 

        # Dual encoders for embeddings and retrieval
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        self.dimension = 512  # CLIP's embedding size
        self.faiss_index = faiss.IndexFlatL2(self.dimension) # FAISS Vector store
        self.metadata_store = {}  # Store mapping of IDs and document page number to content

    # Read a PDF document using PyMuPDF
    # Returns list of page images in cv2 format
    def read_from_path(self, filepath):
        doc = pymupdf.open(filepath)
        return [self.pixmap_to_cv2(page.get_pixmap(dpi=300)) for page in doc]

    # Convert PyMuPDF pixmap to cv2
    def pixmap_to_cv2(self, pixmap):
        bytes = np.frombuffer(pixmap.samples, dtype=np.uint8)
        image = bytes.reshape(pixmap.height, pixmap.width, pixmap.n)
        image = image[..., ::-1]
        return image

    # Takes in image object from read_from_path()
    # Detects layout -> Processes ROI by label
    def detect_layout(self, image):
        layout = self.model.detect(image)

        # Separate boxes by category
        text_blocks = lp.Layout([b for b in layout if b.type=='Text'])
        title_blocks = lp.Layout([b for b in layout if b.type=='Title'])
        list_blocks = lp.Layout([b for b in layout if b.type=='List'])
        table_blocks = lp.Layout([b for b in layout if b.type=='Table'])
        figure_blocks = lp.Layout([b for b in layout if b.type=='Figure'])

        # Processing text blocks
        # Eliminate text blocks nested in images/figures
        text_blocks = lp.Layout([b for b in text_blocks \
                        if not any(b.is_in(b_fig) for b_fig in figure_blocks)])
        # Sort boxes
        h, w = image.shape[:2]
        left_interval = lp.Interval(0, w/2*1.05, axis='x').put_on_canvas(image)
        left_blocks = text_blocks.filter_by(left_interval, center=True)
        left_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)
        # The b.coordinates[1] corresponds to the y coordinate of the region
        # sort based on that can simulate the top-to-bottom reading order 
        right_blocks = lp.Layout([b for b in text_blocks if b not in left_blocks])
        right_blocks.sort(key = lambda b:b.coordinates[1], inplace=True)

        # And finally combine the two lists and add the index
        text_blocks = lp.Layout([b.set(id = idx) for idx, b in enumerate(left_blocks + right_blocks)])

        # Perform OCR to extract text
        for block in text_blocks + title_blocks + list_blocks + table_blocks:
            # Add padding in each image segment to improve robustness
            text = self._ocr_on_block(image, block)
            block.set(text=text, inplace=True) # Assign parsed text to block element

        # Return all blocks on the page as a list
        return text_blocks + title_blocks + list_blocks + table_blocks + figure_blocks

    # Function to crop an image given block's bbox and additional padding
    def _crop_image(self, image, block, padding=10):
        return (block.pad(left=padding, right=padding, top=padding, bottom=padding).crop_image(image))

    # Perform OCR to extract text given image and block (for text, tables and lists)
    def _ocr_on_block(self, image, block):
        # Add padding in each image segment to improve robustness
        segment_image = (block.pad(left=5, right=5, top=5, bottom=5).crop_image(image))
        return self.ocr_agent.detect(segment_image)

    # Vectorstore functions
    # Function to encode text
    def encode_text(self, text):
        inputs = self.clip_processor(text=[text], return_tensors="pt")
        with torch.no_grad():
            embedding = self.clip_model.get_text_features(**inputs).numpy()
        return embedding / np.linalg.norm(embedding)  # Normalize

    # Function to encode image
    def encode_image(self, image):
        inputs = self.clip_processor(images=image, return_tensors="pt")
        with torch.no_grad():
            embedding = self.clip_model.get_image_features(**inputs).numpy()
        return embedding / np.linalg.norm(embedding)  # Normalize

    # Function to add item to FAISS
    def add_to_faiss(self, embedding, content, content_type, page_idx):
        idx = len(self.metadata_store)  # Assign unique index
        self.faiss_index.add(embedding)
        self.metadata_store[idx] = {"type": content_type, "content": content, "page": page_idx}
    
    # Perform retrieval
    # TODO: implement 2 stage retrieval
    def search_faiss(self, query, k=10):
        query_embedding = self.encode_text(query)
        _, indices = self.faiss_index.search(query_embedding, k)
        
        # Display retrieved items
        # retrieved items accessed by metadata_store using fetched indices
        for idx in indices[0]:
            print(f"Retrieved {self.metadata_store[idx]['type']}: {self.metadata_store[idx]['content']}")

In [5]:
# Sample run for 1 document
pipeline = DocumentAnalysis()
doc_path = "../data/1706.03762.pdf"
doc = pipeline.read_from_path(doc_path)

# Processing for each page
# Remove [:3] for entire doc, keep for testing
for page_idx, page in enumerate(doc[:3]):
    blocks = pipeline.detect_layout(page)

    # Processing for each block to be vectorized
    for b in blocks:
        # Process as an image if detected type is a figure, else process as text
        if b.type == "Figure":
            # Crop and get image embeddings
            segmented_image = pipeline._crop_image(page, b, padding=20)
            image_embs = pipeline.encode_image(segmented_image)
            pipeline.add_to_faiss(embedding=image_embs, content="Figure", content_type=b.type, page_idx=page_idx)
        else:
            # Encode using text and add to FAISS
            text_embs = pipeline.encode_text(b.text)
            pipeline.add_to_faiss(embedding=text_embs, content=b.text, content_type=b.type, page_idx=page_idx)
            

Token indices sequence length is longer than the specified maximum sequence length for this model (238 > 77). Running this sequence through the model will result in indexing errors


ValueError: Sequence length must be less than max_position_embeddings (got `sequence length`: 238 and max_position_embeddings: 77

In [ ]:
# Search testing

In [ ]:
# Store sample text and images
# For each text/image/figure block: 
# If text, encode text, if image: encode image, if figure(table): LayoutParser into text and encode text

add_to_faiss(encode_text("Solar panel research"), "Solar panel research", "text")
add_to_faiss(encode_text("Experimental results"), "Experimental results", "text")
add_to_faiss(encode_image("figure1.png"), "figure1.png", "image")
add_to_faiss(encode_image("figure2.png"), "figure2.png", "image")

# Perform retrieval
# To implement 2 stage retrieval
def search_faiss(query, k=3):
    query_embedding = encode_text(query)
    _, indices = faiss_index.search(query_embedding, k)
    
    # Display retrieved items
    for idx in indices[0]:
        print(f"Retrieved {metadata_store[idx]['type']}: {metadata_store[idx]['content']}")

# Example: Retrieve images using text query
search_faiss("solar energy", query_type="text", k=2)

In [ ]:
# Unified function